In [2]:
import dlib
import cv2
import numpy as np
import imutils
from imutils import face_utils


In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

image = cv2.imread("/home/dupmaka/2D+3D/F001/T1/0000.jpg")
# image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# detect faces in the grayscale image
rects = detector(gray, 1)


# loop over the face detections
for (i, rect) in enumerate(rects):
	# determine the facial landmarks for the face region, then
	# convert the facial landmark (x, y)-coordinates to a NumPy
	# array
	shape = predictor(gray, rect)
	shape = face_utils.shape_to_np(shape)
	# convert dlib's rectangle to a OpenCV-style bounding box
	# [i.e., (x, y, w, h)], then draw the face bounding box
	(x, y, w, h) = face_utils.rect_to_bb(rect)
	cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
	# show the face number
	cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
		cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
	# loop over the (x, y)-coordinates for the facial landmarks
	# and draw them on the image
	for (x, y) in shape:
		cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
# show the output image with the face detections + facial landmarks
cv2.imwrite("sample.jpg", image)


True

In [4]:
# get two points, 38, 45 on the left and right eye 
# 49 left mouth, 55 right mouth

image = cv2.imread("/home/dupmaka/2D+3D/F001/T1/0000.jpg")
shape = predictor(gray, rects[0])
shape = face_utils.shape_to_np(shape)

left_eye = shape[37]
right_eye = shape[44]
left_mouth = shape[48]
right_mouth = shape[55]

print(left_eye, right_eye)
print(left_mouth, right_mouth)
print(image.shape)

[377 779] [688 778]
[ 435 1060] [ 610 1085]
(1392, 1040, 3)


In [5]:
color_left = image[left_eye[1], left_eye[0]]
color_right = image[right_eye[1], right_eye[0]]
color_left_mouth = image[left_mouth[1], left_mouth[0]]
color_right_mouth = image[right_mouth[1], right_mouth[0]]
print(color_left, color_right, color_left_mouth, color_right_mouth)

print(image.shape[0], image.shape[1])
left_eye_norm = np.array([left_eye[0]/image.shape[0], left_eye[1]/image.shape[1]])
right_eye_norm = np.array([right_eye[0]/image.shape[0], right_eye[1]/image.shape[1]])
left_mouth_norm = np.array([left_mouth[0]/image.shape[0], left_mouth[1]/image.shape[1]])
right_mouth_norm = np.array([right_mouth[0]/image.shape[0], right_mouth[1]/image.shape[1]])

# print(left_eye_norm, right_eye_norm)

[20 20 32] [20 17 33] [ 46  60 102] [ 48  52 103]
1392 1040


In [6]:
import pyrender, trimesh, cv2, openmesh, os

mesh = openmesh.read_trimesh("/home/dupmaka/2D+3D/F001/T1/0000.obj", vertex_tex_coord=True, face_color=True) 
verts = mesh.points()

tex = []

for i, vh in enumerate(mesh.vertices()):                                                                                                                                    
    tex.append(mesh.texcoord2D(vh))

tex = np.array(tex)
print(tex.shape)
left_eye_repeated = np.repeat(np.array([left_eye]), tex.shape[0], axis=0)
right_eye_repeated = np.repeat(np.array([right_eye]), tex.shape[0], axis=0)
left_mouth_repeated = np.repeat(np.array([left_mouth]), tex.shape[0], axis=0)
right_mouth_repeated = np.repeat(np.array([right_mouth]), tex.shape[0], axis=0)

diff_left = np.linalg.norm(tex-left_eye_repeated, axis=1)
diff_right = np.linalg.norm(tex-right_eye_repeated, axis=1)
diff_left_mouth = np.linalg.norm(tex-left_mouth_repeated, axis=1)
diff_right_mouth = np.linalg.norm(tex-right_mouth_repeated, axis=1)

print(diff_left.shape, diff_right.shape)

left_vert_idx = np.argmin(diff_left)
right_vert_idx = np.argmin(diff_right)
left_mouth_vert_idx = np.argmin(diff_left_mouth)
right_mouth_vert_idx = np.argmin(diff_right_mouth)

print(left_eye, right_eye)

left_vert = verts[left_vert_idx]
right_vert = verts[right_vert_idx]
left_mouth_vert = verts[left_mouth_vert_idx]
right_mouth_vert = verts[right_mouth_vert_idx]

print(left_eye, right_eye)
print(left_vert, right_vert)
print(left_mouth_vert, right_mouth_vert)

(43436, 2)
(43436,) (43436,)
[377 779] [688 778]
[377 779] [688 778]
[  65.58623505   93.16883087 -130.48225403] [  97.90278625   66.4495163  -130.39753723]
[  52.75791168  100.61457825 -131.60823059] [  65.58623505   93.16883087 -130.48225403]


In [10]:
# get the points that corresond to same locations on the facescape mesh model

import pyrender, trimesh, cv2, openmesh, os
import numpy as np
import src.renderer as renderer
import src.camera as camera
import src.utility as util

# read model 
model = trimesh.load_mesh("/home/dupmaka/facescape_old/toolkit/demo_output/original_mesh.obj", process=False)

# get vertices using openmesh, because trimesh doesn't preserve vertex number and order
om_mesh = openmesh.read_trimesh("/home/dupmaka/facescape_old/toolkit/demo_output/original_mesh.obj") 
verts = om_mesh.points()

# set material
# model.visual.material.diffuse = np.array([255, 255, 255, 255], dtype=np.uint8)

# # set K Rt (cv camera coordinate)
# K = np.array([[2000, 0 , 499.5],
#               [0, 2000, 499.5],
#               [0, 0, 1]])
# Rt = np.array([[1, 0 , 0, 0],
#                [0, -1, 0, 0],
#                [0, 0, -1, 600]])

# # render
# _, color = renderer.render_cvcam(model, K, Rt, scale=1.0, 
#                                  rend_size=(1000, 1000), flat_shading=False)

# # read landmark indices, 'v16' is for bilinear model 1.6 and later versions
# lm_list_v16 = np.load("./predef/landmark_indices.npz")['v16']

# # make camera for projection
# cam = camera.CamPara(K = K, Rt = Rt)

# # draw landmarks
# color_draw = color.copy()
# for ind, lm_ind in enumerate(lm_list_v16):
#     uv = cam.project(verts[lm_ind])
#     u, v = np.round(uv).astype(np.int)
#     color_draw = cv2.circle(color_draw, (u, v), 10, (100, 100, 100), -1)
#     color_draw = cv2.putText(color_draw, "%02d"%(ind), (u-8, v+4), 
#                              fontFace = cv2.FONT_HERSHEY_SIMPLEX,
#                              fontScale = 0.4,
#                              color = (255, 255, 255))

# util.show_img_arr(color_draw, bgr_mode = True)

PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex vertex
PolyMeshT::add_face: complex

In [11]:
# points from the bilinear model 
left_eye_fs = [-9.511489705882353318e-02, -1.171094999999999775e-01, 2.978996264705880748e+00]
right_eye_fs = [9.475910294117646793e-02, -1.038524999999999726e-01, 2.956299264705880780e+00] 
left_mouth_fs = [-6.529089705882352990e-02, 8.611350000000000948e-02, 3.009020264705880798e+00]
right_mouth_fs = [6.172010294117646900e-02, 9.618150000000003086e-02, 2.990527264705881372e+00]

In [14]:
# IPC with the sampled points 

# left_vert
# right_vert
# left_eye_fs
# right_eye_fs

dist = left_eye_fs-left_vert

# # bp4d
# mesh

# # facescape
# om_mesh

# set up homography DLT
points = [left_vert, right_vert, left_mouth_vert, right_mouth_vert]
print(points)
img_points = [left_eye_fs, right_eye_fs, left_mouth_fs, right_mouth_fs]
print(img_points)

A = []
for i in range(4):
    # x,y will be image points
    x = points[i][0]
    y = points[i][1]
    
    # x_prime,y_prime will be corresponding points
    x_prime = img_points[i][0]
    y_prime = img_points[i][1]
    
    A_eq_1 = [-x_prime, -y_prime, -1, 0, 0, 0, x*x_prime, x*y_prime, x] 
    A.append(A_eq_1)
    A_eq_2 = [0, 0, 0, -x_prime, -y_prime, -1, y*x_prime, y*y_prime, y]
    A.append(A_eq_2)
    
u, s, vt = np.linalg.svd(A)

H = vt[-1:,:].reshape((3,3))

initial = np.array([[1., 0., 0., dist[0]], [0., 1., 0., dist[1]], [0., 0., 1., dist[2]], [0., 0., 0., 1.]])

matrix, transformed, cost = trimesh.registration.icp(mesh.points(), om_mesh.points(), initial)

[array([  65.58623505,   93.16883087, -130.48225403]), array([  97.90278625,   66.4495163 , -130.39753723]), array([  52.75791168,  100.61457825, -131.60823059]), array([  65.58623505,   93.16883087, -130.48225403])]
[[-0.09511489705882353, -0.11710949999999998, 2.9789962647058807], [0.09475910294117647, -0.10385249999999997, 2.956299264705881], [-0.06529089705882353, 0.08611350000000001, 3.009020264705881], [0.06172010294117647, 0.09618150000000003, 2.9905272647058814]]
[[0.09511489705882353, 0.11710949999999998, -1, 0, 0, 0, -6.238227994912877, -7.680771193164824, 65.58623504638672], [0, 0, 0, 0.09511489705882353, 0.11710949999999998, -1, -8.861743757441465, -10.910955198955534, 93.16883087158203], [-0.09475910294117647, 0.10385249999999997, -1, 0, 0, 0, 9.277180200954437, -10.167449109535214, 97.90278625488281], [0, 0, 0, -0.09475910294117647, 0.10385249999999997, -1, 6.296696555120692, -6.9009483911705, 66.44951629638672], [0.06529089705882353, -0.08611350000000001, -1, 0, 0, 0, -3

ValueError: matrix shape ((3, 3)) doesn't match points ((43436, 3))

In [30]:
import pytorch3d 
from pytorch3d.io import load_obj, save_obj
import torch

# print(type(transformed))
# print(type(om_mesh.faces()))

torch_transformed = torch.from_numpy(transformed)

# print(type(torch_transformed))
vertices, faces, _ = load_obj("/home/dupmaka/facescape_old/toolkit/demo_output/original_mesh.obj")

# print(type(vertices))

pytorch3d.io.save_obj("test.obj", torch_transformed, faces.verts_idx)



An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/home/dupmaka/.conda/envs/l3d/lib/python3.9/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/home/dupmaka/.conda/envs/l3d/lib/python3.9/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt
An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/home/dupmaka/.conda/envs/l3d/lib/python3.9/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/home/dupmaka/.conda/envs/l3d/lib/python3.9/site-packages/iopath/common/event_logger.py", line 97, in log_event
    del self._evt
AttributeError: _evt
